In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


def bar_chart(labels, values, title, xaxis = '', yaxis = '', xsize = 14, subtitle=''):
    d = [go.Bar(
            x=labels,
            y=values,
            text=values,
            textposition = 'outside',
            opacity=1)]

    layout = go.Layout(
        title='<b>'+title+'</b><br>'+subtitle,
        xaxis=dict(
            title=xaxis,
            tickfont=dict(
                size=xsize,
                color='rgb(107, 107, 107)'
            )
        ),
        yaxis=dict(
            title=yaxis,
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
    )

    fig = go.Figure(data=d, layout=layout)
    iplot(fig, filename='color-bar')
    
def viz(groupby_df, title = '', xaxis = '', yaxis = '', xsize = 14, subtitle=''):
    labels = list(groupby_df.index)
    values = list(groupby_df.values)
    
    d = [go.Bar(
            x=labels,
            y=values,
            text=values,
            textposition = 'outside',
            opacity=1)]

    layout = go.Layout(
        title='<b>'+title+'</b><br>'+subtitle,
        xaxis=dict(
            title=xaxis,
            tickfont=dict(
                size=xsize,
                color='rgb(107, 107, 107)'
            )
        ),
        yaxis=dict(
            title=yaxis,
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
    )

    fig = go.Figure(data=d, layout=layout)
    iplot(fig, filename='color-bar')

In [4]:
data = pd.read_excel('CoC__Departments_Research_TK_030918_Edits.xlsx', encoding = 'iso-8859-1')

In [5]:
data.columns

Index(['index', 'City Department', 'Webpage Title', 'URL', 'Persona', 'Verb',
       'SVC in URL?', 'Button Present?', 'i_want_to', 'nav',
       'No. of E-mail Addresses', 'E-mail Address ', 'PDF Count',
       'External Link Count', 'External Link URL(s)', 'Top Words'],
      dtype='object')

In [6]:
columns = ['index', 'dept', 'title', 'url', 'persona', 'verb',
       'service', 'button', 'i_want_to', 'nav',
       'email_count', 'emails', 'pdf_count',
       'external_count', 'external_urls', 'top_words']
data.columns = columns

In [7]:
data1 = data[(data['dept'] == 'Department of Business Affairs and Consumer Protection') | (data['dept'] == 'Department of Finance')].set_index(keys='index').reset_index(drop=True)
data1 = data1.fillna('NA')
data1['verb'] = data1.verb.apply(lambda x: x.strip())

In [8]:
data1.verb.unique()

array(['Apply', 'Pay', 'View', 'Register', 'File', 'Learn', 'Multiple',
       'Obtain', 'Enroll', 'Request', 'Find', 'Report', 'Other', 'NA'], dtype=object)

In [9]:
verbs = set()
for words in data1.verb.unique():
    for wor in re.split('W+ |;', words):
        verbs.add(wor.strip())
verbs

{'Apply',
 'Enroll',
 'File',
 'Find',
 'Learn',
 'Multiple',
 'NA',
 'Obtain',
 'Other',
 'Pay',
 'Register',
 'Report',
 'Request',
 'View'}

In [10]:
data1['verb_list'] = data1.verb.apply(lambda x: re.split('W+ | ;|; |,|;',x.strip()))

In [11]:
data1['verb_cnt'] = data1.verb_list.apply(lambda x: len(x))

In [12]:
data1.groupby('verb_cnt').size()

verb_cnt
1    604
dtype: int64

In [13]:
data_single = data1[data1['verb_cnt'] == 1]
business = data_single[data_single.dept == 'Department of Business Affairs and Consumer Protection']
finance = data_single[data_single.dept == 'Department of Finance']
business_df = business.groupby('verb').size().sort_values(ascending=False)
finance_df = finance.groupby('verb').size().sort_values(ascending=False)
viz(business_df,'Number of Webpages by Verb for Department of Business Affairs and Consumer Protection','Verb','Number of Webpages')
print('Based on manual categorization, BACP pages were categorized into the above verb categories.')
viz(finance_df,'<b>Number of Webpages by Verb for Department of Finance', 'Verb', 'Number of Webpages',12)
print('Based on manual categorization, DoF pages were categorized into the above verb categories.')

Based on manual categorization, BACP pages were categorized into the above verb categories.


Based on manual categorization, DoF pages were categorized into the above verb categories.


## Webpages Per Agency

In [407]:
web_per_dept = data.groupby('dept').size().sort_values(ascending=False)
values = list(web_per_dept.values)
depts = list(web_per_dept.index)

In [554]:
bar_chart(depts[1:], values[1:], 'Webpages per Department','Department', 'Number of Webpages',8)

## Personas

In [537]:
b_personas = {}
for row in business.persona.apply(lambda x: re.split(';',x)):
    for persona in row:
        if persona not in b_personas:
            b_personas[persona] = 1
        else:
            b_personas[persona] += 1
f_personas = {}    
for row in finance.persona.apply(lambda x: re.split(';',x)):
    for persona in row:
        if persona not in f_personas:
            f_personas[persona] = 1
        else:
            f_personas[persona] += 1

In [566]:
f_list = list(f_personas.items())
b_list = list(b_personas.items())
f_list.sort(key=lambda x: x[1])
f_list = f_list[::-1]
b_list.sort(key=lambda x: x[1])
b_list = b_list[::-1]
f_persona_list = [x for x,y in f_list]
f_count_list = [y for x,y in f_list]
b_persona_list = [x for x,y in b_list]
b_count_list = [y for x,y in b_list]
bar_chart(f_persona_list[:20],f_count_list[:20],"Dept. of Finance Persona Appearances",xsize=11)
bar_chart(b_persona_list[:20],b_count_list[:20],"Dept. of Business Affairs and Consumer<br>Protection Persona Appearances",xsize=11)

In [2]:
import os
os.environ

environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\vanguiano\\AppData\\Roaming', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'DESKTOP-000023', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\vanguiano', 'LOCALAPPDATA': 'C:\\Users\\vanguiano\\AppData\\Local', 'LOGONSERVER': '\\\\DESKTOP-000023', 'NUMBER_OF_PROCESSORS': '4', 'ONEDRIVE': 'C:\\Users\\vanguiano\\OneDrive', 'OS': 'Windows_NT', 'PATH': 'C:\\Users\\vanguiano\\Anaconda3;C:\\Users\\vanguiano\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\vanguiano\\Anaconda3\\Library\\usr\\bin;C:\\Users\\vanguiano\\Anaconda3\\Library\\bin;C:\\Users\\vanguiano\\Anaconda3\\Scripts;C:\\Users\\vanguiano\\Anaconda3\\Library\\bin;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\P